# Build Chatbot
```11_Chatbot.ipynb```
- https:


In [13]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

================================== Ai Message ==================================

Hi, Bob!


In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [16]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Hi!, I am bob.', additional_kwargs={}, response_metadata={}, id='44094113-cad0-46d8-a3c7-96c5ae397c73'),
  AIMessage(content='Hello bob. how can I help you.', additional_kwargs={}, response_metadata={}, id='ed5b2aad-53b9-445c-906d-dbdca8c1323d'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='fdd8cb9e-dd36-4ae7-8af3-941a85e08c3d'),
  AIMessage(content='Hi, Bob! Nice to meet you.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 34, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDLLlbfHFQ6DShDA4rEmOvaJwcvaE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}

In [17]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

87f7ddf8-8c56-11f0-8169-9cb1507ac86b


{'messages': [HumanMessage(content='say my name.', additional_kwargs={}, response_metadata={}, id='2b98abec-d5ed-4676-af36-7193a468196f'),
  AIMessage(content="I'm sorry, but I don't know your name. Could you please tell me?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 11, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDLLoeQZpwCYf9xMnG5fPEejavl9N', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1917e1ce-f4c5-4503-95f9-35f750e65442-0', usage_metadata={'input_tokens': 11, 'output_tokens': 16, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

## Langgraph + PromptTemplate

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


## State 확장

In [ ]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [23]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='Hi. I am bob')],
    'lang': 'Spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

Hi. I am bob
================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿En qué puedo ayudarte hoy?


## 대화 기록 관리하기
대화 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림. 

In [36]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',      # 최신 메세지들을
    max_tokens=65,         # 최대 65토큰 까지만 허용,
    token_counter=llm,    # llm 모델에 맞춰서 토큰 세고
    include_system=True,  # system 프롬프트는 포함(정리X)
    allow_partial=False,  # 메세지 중간에서 자르지는 말고
    start_on='human',     # 잘린 메세지의 첫번째는 사람 메세지가 되도록
    
)

messages = [
    SystemMessage(content="you're a good assistant"),

    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)  # 시스템메시지 -> 2+2 부터 등장


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [38]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    # print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    # print('정리 후 메시지 개수: ', len(trimmed_messages))
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': [res]}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [39]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='내 상태는 어때')],
    'lang': '한국어'
}

for chunk, metadata in graph.stream(state, config, stream_mode='messages'):
    print(chunk.content, end='|')

|현재| 당신|의| 상태|에| 대해| 구|체|적인| 정보|가| 없|어서| 정확|히| 말씀|드|리|기| 어렵|습니다|.| 몸| 상태|,| 감|정| 상태|,| 또는| 다른| 어떤| 면|에| 대해| 말씀|해|주|시면| 더| 도움|을| 드|릴| 수| 있을| 것| 같|아요|.| 어떤| 부분|이| 걱|정|되|거나| 궁|금|하|신|지| 알려|주세요|.||